In [1]:
#Importing Libraries
from keras.layers import Input,Dense,Flatten
from keras.models import Model
from tensorflow.keras import layers
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob

In [2]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [3]:
train_path = '../input/100-bird-species/train'
valid_path = '../input/100-bird-species/valid'
test_path = '../input/100-bird-species/test'

In [4]:
# add preprocessing layer to the front of EfficientNet
xcept = Xception(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

2022-03-26 12:36:17.817318: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-26 12:36:17.893007: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-26 12:36:17.893715: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-26 12:36:17.894895: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

83697664/83683744 [==============================] - 0s 0us/step


In [5]:
# don't train existing weights
for layer in xcept.layers:
  layer.trainable = False

In [6]:
#Getting Number of Categories
folders = glob('../input/100-bird-species/train/*')

In [7]:
x = Flatten()(xcept.output)

In [8]:
x = layers.Dense(256, 'relu', kernel_initializer='he_normal')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

In [9]:
prediction = Dense(len(folders), activation='softmax')(x)

In [10]:
# create a model object
model = Model(inputs=xcept.input, outputs=prediction)

In [11]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [12]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [13]:
#Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

valid_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('../input/100-bird-species/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
valid_set = valid_datagen.flow_from_directory('../input/100-bird-species/valid',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('../input/100-bird-species/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 58388 images belonging to 400 classes.
Found 2000 images belonging to 400 classes.
Found 2000 images belonging to 400 classes.


In [2]:
#Fitting the model
r = model.fit(
  training_set,
  validation_data=valid_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(valid_set)
)


2022-03-26 12:36:29.082684: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes
are enabled (registered 2)

Epoch 1/10

2022-03-26 12:36:32.884303: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005

1825/1825 [==============================] - 802s 435ms/step - loss: 1.7999 - accuracy: 0.5967 - val_loss: 0.5375 - val_accuracy:
0.8500
Epoch 2/10
1825/1825 [==============================] - 700s 383ms/step - loss: 0.8449 - accuracy: 0.7702 - val_loss: 0.4163 - val_accuracy:
0.8780
Epoch 3/10
1825/1825 [==============================] - 711s 389ms/step - loss: 0.7021 - accuracy: 0.8051 - val_loss: 0.3796 - val_accuracy:
0.8840
Epoch 4/10
1825/1825 [==============================] - 715s 392ms/step - loss: 0.6223 - accuracy: 0.8249 - val_loss: 0.3033 - val_accuracy:
0.9115
Epoch 5/10
1825/1825 [==============================] - 730s 400ms/step - loss: 0.5600 - accuracy: 0.8399 - val_loss: 0.2960 - val_accuracy:
0.9

In [15]:
#Saving the Model.
import tensorflow as tf

from keras.models import load_model

model.save('./bird_classification_new_model.h5')

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [1]:
model.evaluate(test_set)

63/63 [==============================] - 13s 201ms/step - loss: 0.1070 - accuracy: 0.9490
[0.10677755026817322, 0.9489999895095825]
